In [1]:
# In summary I will try to use feature that selected with model related to Decision tree
# so I decide to use feature with ExtraTree and RandomForest
# Summary #1

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import Ridge
import pandas as pd
import numpy as np
from datetime import datetime
import datetime
from sklearn.model_selection import KFold
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# prepare data
def create_prenext_week(data):
    dataset = data.copy()
    dataset['preweek'] = dataset['Week']-1
    dataset.loc[dataset['preweek'] == 0, 'preweek'] = 52
    dataset['nextweek'] = dataset['Week']+1
    dataset.loc[dataset['nextweek'] == 53, 'nextweek'] = 1
    PreNext1week = dataset.copy()
    dataset['pre2week'] = dataset['preweek']-1
    dataset.loc[dataset['pre2week'] == 0, 'pre2week'] = 52
    dataset['next2week'] = dataset['nextweek']+1
    dataset.loc[dataset['next2week'] == 53, 'next2week'] = 1
    PreNext2week = dataset.copy()
    return(PreNext1week,PreNext2week)
def get_holiday(date):
    super_bowl = ['2010-02-12','2011-02-11','2012-02-10','2013-02-08']
    labor = ['2010-09-10','2011-09-09','2012-09-07']
    thank = ['2010-11-26','2011-11-25','2012-11-23']
    chris = ['2010-12-31','2011-12-30','2012-12-28']
    columbus =['2010-10-15','2011-10-14','2012-10-12']
    veterans = ['2010-11-12','2011-11-11','2012-11-16']
    independence = ['2010-7-9','2011-7-8','2012-7-6','2013-7-5']
    memorial = ['2010-6-4','2011-6-3','2012-6-1','2013-5-31']
    washington = ['2010-2-19','2011-2-25','2012-2-24','2013-2-22']
    if date in super_bowl:
        return [0,0,0,0,0,0,0,0,1]
    elif date in labor:
        return [0,0,0,0,0,0,0,1,0]
    elif date in thank:
        return [0,0,0,0,0,0,1,0,0]
    elif date in chris: 
        return [0,0,0,0,0,1,0,0,0]
    elif date in columbus: 
        return [0,0,0,0,1,0,0,0,0]
    elif date in veterans:
        return [0,0,0,1,0,0,0,0,0]
    elif date in independence:
        return [0,0,1,0,0,0,0,0,0]
    elif date in memorial:
        return [0,1,0,0,0,0,0,0,0]
    elif date in washington:
        return [1,0,0,0,0,0,0,0,0]
    else:
        return [0,0,0,0,0,0,0,0,0]
def multi_holiday(data):
    dataset = data.copy()
    holiday = []
    dates = data['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday.append(get_holiday(str(dates[i])))
    holiday = pd.DataFrame(holiday)
    holiday.columns = ['super_bowl', 'labor','thank','chris','columbus','veterans','independence','memorial','washington']
    dataset = pd.merge(dataset, holiday, left_index=True, right_index=True)
    normal_holiday = dataset.copy()
    holiday_pre1 = []
    dates = data['Date'].apply(lambda x: (x-datetime.timedelta(days=7)).strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday_pre1.append(get_holiday(str(dates[i])))
    holiday_next1 = []
    dates = data['Date'].apply(lambda x: (x+datetime.timedelta(days=7)).strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday_next1.append(get_holiday(str(dates[i])))
    holiday_pre1 = pd.DataFrame(holiday_pre1)
    holiday_next1 = pd.DataFrame(holiday_next1)
    holiday_pre1.columns = ['super_bowl_p1', 'labor_p1','thank_p1','chris_p1','columbus_p1','veterans_p1','independence_p1','memorial_p1','washington_p1']
    holiday_next1.columns = ['super_bowl_n1', 'labor_n1','thank_n1','chris_n1','columbus_n1','veterans_n1','independence_n1','memorial_n1','washington_n1']
    dataset = pd.merge(dataset, holiday_pre1, left_index=True, right_index=True)
    dataset = pd.merge(dataset, holiday_next1, left_index=True, right_index=True)
    PreNext1week = dataset.copy()
    holiday_pre2 = []
    dates = data['Date'].apply(lambda x: (x-datetime.timedelta(days=7)).strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday_pre2.append(get_holiday(str(dates[i])))
    holiday_next2 = []
    dates = data['Date'].apply(lambda x: (x+datetime.timedelta(days=7)).strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday_next2.append(get_holiday(str(dates[i])))
    holiday_pre2 = pd.DataFrame(holiday_pre2)
    holiday_next2 = pd.DataFrame(holiday_next2)
    holiday_pre2.columns = ['super_bowl_p2', 'labor_p2','thank_p2','chris_p2','columbus_p2','veterans_p2','independence_p2','memorial_p2','washington_p2']
    holiday_next2.columns = ['super_bowl_n2', 'labor_n2','thank_n2','chris_n2','columbus_n2','veterans_n2','independence_n2','memorial_n2','washington_n2']
    dataset = pd.merge(dataset, holiday_pre2, left_index=True, right_index=True)
    dataset = pd.merge(dataset, holiday_next2, left_index=True, right_index=True)
    PreNext2week = dataset.copy()
    return(normal_holiday,PreNext1week,PreNext2week)

In [4]:
def ET_Realtest(name,train,test):
    name = './'+name+'_ET.csv'
    f = open(name,'w')
    f.write('Id,Weekly_Sales\n')
    for store in test.Store.unique():
        select_store_test = test.loc[test['Store'] == store]
        select_store_train = train.loc[train['Store'] == store]
        for dept in select_store_test.Dept.unique():
            select_dept_test = select_store_test.loc[select_store_test['Dept'] == dept]
            dates = select_dept_test['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).values
            select_dept_test = select_dept_test.drop(columns=['Store', 'Dept','Date']).reset_index()
            select_dept_train = select_store_train.loc[select_store_train['Dept'] == dept]
            select_dept_train = select_dept_train.drop(columns=['Store', 'Dept','Date']).reset_index()
            if len(select_dept_train.index) > 0: # handle no train data for test dept ex. store 5 dept 99
                X_train = select_dept_train.drop(columns=['Weekly_Sales'])
                Y_train = select_dept_train['Weekly_Sales']
                X_test = select_dept_test
                clf = ExtraTreesRegressor()
                clf.fit(X_train,Y_train)
                test_y = clf.predict(X_test)
                for i in range(len(test_y)):
                    Id = str(store)+'_'+str(dept)+'_'+str(dates[i])
                    sales = test_y[i]
                    f.write('%s,%s\n'%(Id,sales))
            else:
                for i in range(len(select_dept_test)):
                    Id = str(store)+'_'+str(dept)+'_'+str(dates[i])
                    sales = 0
                    f.write('%s,%s\n'%(Id,sales))
    f.close()
def RF_Realtest(name,train,test):
    name = './'+name+'_RF.csv'
    f = open(name,'w')
    f.write('Id,Weekly_Sales\n')
    for store in test.Store.unique():
        select_store_test = test.loc[test['Store'] == store]
        select_store_train = train.loc[train['Store'] == store]
        for dept in select_store_test.Dept.unique():
            select_dept_test = select_store_test.loc[select_store_test['Dept'] == dept]
            dates = select_dept_test['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).values
            select_dept_test = select_dept_test.drop(columns=['Store', 'Dept','Date']).reset_index()
            select_dept_train = select_store_train.loc[select_store_train['Dept'] == dept]
            select_dept_train = select_dept_train.drop(columns=['Store', 'Dept','Date']).reset_index()
            if len(select_dept_train.index) > 0: # handle no train data for test dept ex. store 5 dept 99
                X_train = select_dept_train.drop(columns=['Weekly_Sales'])
                Y_train = select_dept_train['Weekly_Sales']
                X_test = select_dept_test
                clf = RandomForestRegressor()
                clf.fit(X_train,Y_train)
                test_y = clf.predict(X_test)
                for i in range(len(test_y)):
                    Id = str(store)+'_'+str(dept)+'_'+str(dates[i])
                    sales = test_y[i]
                    f.write('%s,%s\n'%(Id,sales))
            else:
                for i in range(len(select_dept_test)):
                    Id = str(store)+'_'+str(dept)+'_'+str(dates[i])
                    sales = 0
                    f.write('%s,%s\n'%(Id,sales))
    f.close()
def ET_Onemodel(name,train,test):
    name = './'+name+'_OneET.csv'
    f = open(name,'w')
    f.write('Id,Weekly_Sales\n')
    dates = test['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).values
    store = test['Store']
    dept = test['Dept']
    test = test.drop(columns=['Date'])
    train = train.drop(columns=['Date'])
    X_train = train.drop(columns=['Weekly_Sales'])
    Y_train = train['Weekly_Sales']
    X_test = test
    clf = ExtraTreesRegressor()
    clf.fit(X_train,Y_train)
    test_y = clf.predict(X_test)
    for i in range(len(test_y)):
        Id = str(store[i])+'_'+str(dept[i])+'_'+str(dates[i])
        sales = test_y[i]
        f.write('%s,%s\n'%(Id,sales))
    f.close()
def RF_Onemodel(name,train,test):
    name = './'+name+'_OneRF.csv'
    f = open(name,'w')
    f.write('Id,Weekly_Sales\n')
    dates = test['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).values
    store = test['Store']
    dept = test['Dept']
    test = test.drop(columns=['Date'])
    train = train.drop(columns=['Date'])
    X_train = train.drop(columns=['Weekly_Sales'])
    Y_train = train['Weekly_Sales']
    X_test = test
    clf = RandomForestRegressor()
    clf.fit(X_train,Y_train)
    test_y = clf.predict(X_test)
    for i in range(len(test_y)):
        Id = str(store[i])+'_'+str(dept[i])+'_'+str(dates[i])
        sales = test_y[i]
        f.write('%s,%s\n'%(Id,sales))
    f.close()

In [ ]:
train = pd.read_csv('./train.csv')
feature = pd.read_csv('./features.csv')
test = pd.read_csv('./test.csv')
train['Date'] = pd.to_datetime(train['Date'])
train['Week'] = train['Date'].dt.week
test['Date'] = pd.to_datetime(test['Date'])
test['Week'] = test['Date'].dt.week
feature['Date'] = pd.to_datetime(feature['Date'])
train['IsHoliday'] = (train[['IsHoliday']]== True).astype(int)
test['IsHoliday'] = (test[['IsHoliday']]== True).astype(int)
train_with_feature = pd.merge_asof(train, feature, on='Store',by='Date')
test_with_feature = pd.merge_asof(test, feature, on='Store',by='Date')

In [ ]:
train = train_with_feature.fillna(train_with_feature.mean())
test = test_with_feature.fillna(test_with_feature.mean())
start_feature_train = train[['Store','Dept','Date','Weekly_Sales','Week','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']]
start_feature_test = test[['Store','Dept','Date','Week','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']]
train = train_with_feature.fillna(train_with_feature.mean())
test = test_with_feature.fillna(test_with_feature.mean())
train1week,train2week = create_prenext_week(start_feature_train)
test1week,test2week = create_prenext_week(start_feature_test)
normaltrain,train1week,train2week = multi_holiday(train2week)
normaltest,test1week,test2week = multi_holiday(test2week)
ET_Realtest('Final',train2week,test2week) #3499.8658
RF_Realtest('Final',train2week,test2week) #4313.12796

ET_Onemodel('Final',train2week,test2week) #4633.38313
RF_Onemodel('Final',train2week,test2week) #3983.31726

In [ ]:
# ExtraTree provide best score 3499.8658